### Step 1: Configure notebook

This notebook might stall when trying to mount the google drive.  It will prompt for an authentication key.  I'm not sure how often it expires.


### Step 2: Prepare embeddings

There are 3 index types / embeddings:
+ Words
  + These are imported from Glove and minimal formatting is applied.
+ Characters
  + These are trained in the model.  However, the dictionary of acceptable values is determined at this stage from `string.printable`
+ Casing
  + A dictionary is initialized at this stage from hard-coded options.
+ Labels
  + The ontonotes training data is used as the source of possible labels.  In addition the 'TITLE' tags are also added to the labels dictionary.


### Step 3: Process data

There are three distinct varieties of data:
+ Original Ontonotes data
  + This data will be used for pre-training the model.  All 3 sets (train, dev, test) are used.
  + The data originates in a 4-column CoNLL format.  We are only interested in the token and label columns.  Each line is a token.  Sentence divisions are indicated by an empty line.
  + Train: for pre-training the model
  + Dev: for determining when the model is trained
  + Test: for establishing the generalized accuracy of the model
+ Domain data
  + This is the manually annotated data that we will use to fine-tune and test our model.
  + It follows a similar format to Ontonotes (4-column, with column order preserved), however the middle two columns have been replaced with 'company' and 'director'
+ BILOU formatted data
  + There are versions of the ontonotes and domain data in this format.
  + It is 2-column, (array_of_tokens, array_of_labels).  Each line is a sentence.

The data needs to be formatted into 4 vectors per sample (word, char, casing, label), where each sample is a sentence.  Each of these vectors needs to be truncated / padded.

There is a caching function that can be used to store the very large (~7GB) vectorized data files.

### Step 4: Build / Train Model

### Step 5: Fine-tune model


# Configuration

In [1]:
import os
import time
import sys
import csv
import pandas as pd
import numpy as np
import string
import ast
from IPython.display import display

from shutil import copyfile
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

csv.field_size_limit(sys.maxsize)

from keras.layers import TimeDistributed, Conv1D, Dense, Embedding, Input, Dropout, LSTM, Bidirectional, MaxPooling1D, Flatten, concatenate
from keras.initializers import RandomUniform
from keras.optimizers import SGD, Nadam
from keras.models import Model, load_model, Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

np.random.seed(1492)

Mounted at /content/drive


Using TensorFlow backend.


Found GPU at: /device:GPU:0


In [0]:
### File configurations
drive_dir = "/content/drive/My Drive/W266_Project/"
data_src = os.path.join(drive_dir,"data")
embed_src = os.path.join(drive_dir,"embeddings")

# cache store
cache_dir = os.path.join(drive_dir, "cache")
embed_store =  os.path.join(cache_dir, 'embed.h5')

### Model Parameters
DROPOUT = 0.2
RECURRENT_DROPOUT = 0.25
CHAR_VOCAB = len(string.printable)
CHAR_EMBEDDING_DIM = 30
WORD_LENGTH = 52
CONV_SIZE = 3
CONV_FILTERS = 30
CONV_STRIDE = 1
CONV_WINDOW = 52
LSTM_STATE_SIZE = 200

EPOCHS = 20
BATCH_SIZE = 400
TRAINING_SIZE = 600000

# embedding to use
# 50d vector is consistent with paper
embedding_file = "glove.6B.50d.txt"

annotation_type = "BILOU"

# data files
train_file = None
dev_file = None
test_file = None
real_file = None

if annotation_type == "BIO":
  train_file = 'onto.train.ner'
  dev_file = 'onto.development.ner'
  test_file = 'onto.test.ner'
  real_file = 'bios-tagged-final-flat.csv'
elif annotation_type == "BILOU":
  train_file = 'onto.train.ner_bilou.csv'
  dev_file = 'onto.development.ner_bilou.csv'
  test_file = 'onto.test.ner_bilou.csv'
  real_file = 'bios-tagged-final-flat_bilou.csv'
else:
  raise Exception(f"unknown annotation: {annotation_type}")

real_training_sizes = [10,25,50,100,200,300]
# whether to freeze the weights on the bilstm
FROZEN = False
# which layer weights to copy over for fine_tuning
# None results in all weights, otherwise provide an array of layer names
FINE_TUNE_LAYERS = ['biLSTM', 'softmax']

model_dir = os.path.join(drive_dir, 'output')

# if loading a pre-trained model set these
model_name = ""
model_load_path = os.path.join(model_dir, model_name + '.h5')
INITIAL_EPOCH = EPOCHS

# else use these
model_name = f"std_400b_glove50d_baseline_{annotation_type}"
model_path = os.path.join(model_dir, model_name + '_{epoch:02d}-{val_loss:.4f}.h5')



### Preprocessing Parameters
UNK_WORD = "<UNK-WORD>"
PAD_WORD = "<PAD-WORD>"

UNK_CHAR = "<UNK-CHAR>"
PAD_CHAR = "<PAD-CHAR>"

# max number of words in a sentence, pad to this length, might throw an error if the sentence is longer
SENTENCE_WIDTH = 256
# max number of characters in a word, pad to this length, will truncate if word is too long
WORD_WIDTH = 52
# symbols to map padding to
CHAR_PAD_SYMBOL = PAD_CHAR
LABEL_PAD_SYMBOL = 'O'
CASE_PAD_SYMBOL = 'other'

# Setup

In [3]:
# show files
print(os.listdir(data_src))
print(os.listdir(embed_src))
print(os.listdir(cache_dir))

['onto.development.ner.sample', 'onto.development.ner', 'onto.test.ner.sample', 'onto.train.ner.sample', 'onto.test.ner', 'onto.train.ner', 'bios-tagged_bilou.csv', 'bios-tagged-set1.csv', 'bios-tagged-set2.csv', 'bios-tagged-final-flat.csv', 'bios-tagged-final-agg.csv', 'onto.train.ner_bilou.csv', 'onto.development.ner_bilou.csv', 'onto.test.ner_bilou.csv', 'bios-tagged-final-flat_bilou.csv']
['glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.6B.50d.txt', 'readme.md', 'Skip100']
['embed.h5', 'onto_train_ner_bilou_word.npy', 'onto_train_ner_bilou_label.npy', 'onto_train_ner_bilou_case.npy', 'onto_train_ner_bilou_char.npy', 'onto_development_ner_bilou_word.npy', 'onto_development_ner_bilou_case.npy', 'onto_development_ner_bilou_label.npy', 'onto_development_ner_bilou_char.npy', 'onto_test_ner_bilou_word.npy', 'onto_test_ner_bilou_case.npy', 'onto_test_ner_bilou_label.npy', 'onto_test_ner_bilou_char.npy', 'bios-tagged_bilou_word.npy', 'bios-tagged_bilou_char.npy', 'bi

# Preprocessing

## Load Embeddings

In [0]:
# consider using tf.nn.embedding_lookup instead
# or maybe nltk.tokenize


def get_casing_ix(word):
  '''
  determines the casing of the word
  
  returns casing_ix
  '''
  if word.istitle():
    return case_to_ix['title']
  elif word.islower():
    return case_to_ix['lower']
  elif word.isupper():
    return case_to_ix['upper']
  elif word.isnumeric():
    return case_to_ix['numeric']
  return case_to_ix['other']

def get_word_ix(word):
  '''
  takes w and returns the index of the word embedding
  out of vocabulary terms return the UNK_WORD and the character embeddings
  
  returns word_ix
  '''
  w = word.lower()
  w_ix = word_to_ix.get(w)
  if w_ix is not None:
    return w_ix
  return word_to_ix[UNK_WORD]

def get_char_ix(char):
  char_ix = char_to_ix.get(char)
  if char_ix is not None:
    return char_ix
  return char_to_ix[UNK_CHAR]
  
def create_character_embeddings(words_df):
  '''
  Optional function to create pre-trained character embeddings from averaged word embeddings.  In the model we generate them from a uniform random distribution and train.
  '''
  characters = {}
  for i, word_vec in enumerate(words_df.reset_index().values):
    for char in word_vec[0]:
      if char in characters:
        characters[char] = [characters[char][0] + word_vec[1:].astype(float), characters[char][1] + 1]
      else:
        characters[char] = [word_vec[1:].astype(float), 1]

  for key in characters:
    characters[key] = np.round(characters[key][0]/characters[key][1],6)
    
def initialize_word_embeddings(file_name, use_cache=True, debug=True, save_cache=True):
  loaded = False
  df = None
  
  if use_cache:
    try:
      print("Attempting to load from cache")
      with pd.HDFStore(embed_store, 'r') as store:
        words = store[file_name]
      words = pd.read_hdf(embed_store, file_name)
      loaded=True
      print("Loaded successfully")
    except:
      print("Cache loading failed")
      loaded=False
  
  if not loaded:
    words = pd.read_csv(os.path.join(embed_src, embedding_file), sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
    # some embeddings come back with word == NaN
    words = words[~words.index.isnull()]
    # add entries for special tokens
    words.loc[UNK_WORD] = [0 for x in words.columns]
    words.loc[PAD_WORD] = [0 for x in words.columns]
    if save_cache:
      with pd.HDFStore(embed_store, 'a') as store:
        store[file_name] = words
  
  word2ix = {word:i for i,word in enumerate(words.index)}
  ix2word = {i:word for i,word in enumerate(words.index)}
  words = words.to_numpy().astype(float)
  
  return words, word2ix, ix2word

def initialize_character_embeddings(vocab=string.printable):
  characters = [x for x in string.printable]
  characters += [UNK_CHAR, PAD_CHAR]
  char2ix = {ch:i for i, ch in enumerate(characters)}
  ix2char = {i:ch for i, ch in enumerate(characters)}
  
  return characters, char2ix, ix2char

def initialize_case_embeddings(vocab=['upper','lower','title','numeric','other']):
  case2ix = {case:i for i, case in enumerate(vocab)}
  ix2case = {}
  cases = []
  for k,v in case2ix.items():
    this_case = np.zeros(len(case2ix))
    this_case[v] = 1
    cases.append(this_case)
    ix2case[v] = k
  cases = np.array(cases)
  
  return cases, case2ix, ix2case

  
def initialize_labels(file_name, annotation):
  label_list = None
  
  if annotation == "BIO":
    domain_labels =  ['B-TITLE', 'I-TITLE']
    data = pd.read_csv(os.path.join(data_src, file_name), sep="\t",  quoting=csv.QUOTE_NONE, header=None, skip_blank_lines=False, engine='python', names =['token', 'pos', 'tree', 'label'])
    data.dropna(subset=['label'], inplace=True)
    label_list = list(data.label.unique())
    label_list += [label for label in domain_labels if label not in label_list]
  elif annotation == "BILOU":
    domain_labels = ['B-TITLE', 'I-TITLE', 'L-TITLE', 'U-TITLE']
    data = pd.read_csv(os.path.join(data_src, file_name), header=0, index_col=0,skip_blank_lines=False, engine='python')
    label_list = list(np.unique(np.concatenate(data.y.apply(lambda x: np.array(ast.literal_eval(x))).values)))
    label_list += [label for label in domain_labels if label not in label_list]
  else:
    raise Exception(f"unknown annotation: {annotation}")
    
  label2ix = {label:i for i, label in enumerate(label_list)}
  ix2label = {i:label for i, label in enumerate(label_list)}
  return label_list, label2ix, ix2label

In [5]:
# load embeddings and format
words, word_to_ix, ix_to_word = initialize_word_embeddings(embedding_file, use_cache=True)
characters, char_to_ix, ix_to_char = initialize_character_embeddings()
cases, case_to_ix, ix_to_case = initialize_case_embeddings()
labels, label_to_ix, ix_to_label = initialize_labels(real_file, annotation_type)

Attempting to load from cache
Loaded successfully


## Process Data

In [0]:
def checkPrior(blah):
  if (blah.prior is None or blah.prior is np.NaN) and (blah.prior_pos is None or blah.prior_pos is np.NaN):
    return True
  else:
    return False
  
def phrase2char(w_vec):
  '''
  This function transforms a sequence of words in index format to a 2d array of character indexes
  
  w_vec - an iterable of word indexes
  
  returns np.ndarray of size (len(w_vec), WORD_WIDTH)
  '''
  phrase_vector = []
  for w_ix in w_vec:
    char_vector = []
    if w_ix not in (word_to_ix[PAD_WORD],word_to_ix[UNK_WORD]):
      for char in ix_to_word[w_ix]:
        char_vector.append(get_char_ix(char))
    phrase_vector.append(np.array(char_vector))
  return pad_sequences(phrase_vector, value=char_to_ix[PAD_CHAR], maxlen=WORD_WIDTH, padding='post')

def pad_truncate(x,width,pad_token):
  if(len(x) > width):
    print(f"Truncating input: {[ix_to_word[ix] for ix in x]}")
    x = x[:256]
  return np.pad(x,pad_width=(0,width-len(x)), mode='constant', constant_values=pad_token)

def verbosity(str, verbose):
  if verbose:
    print(str)

def sent_to_casing_ix(words):
  sentence_vector = []
  for word in words:
    sentence_vector.append(get_casing_ix(word))
  return sentence_vector

def sent_to_word_ix(words):
  sentence_vector = []
  for word in words:
    sentence_vector.append(get_word_ix(word))
  return sentence_vector

def sent_to_label_ix(labels):
  label_vector = []
  for label in labels:
    label_vector.append(label_to_ix[label])
  return label_vector

def preprocess_data(file_name, annotation, use_cache=True, debug=True, save_cache=True):
  '''
  Prepares data for model.  It can be used for both training and test data.
  
  returns pd.DataFrame
  '''
  clean_name = os.path.join(cache_dir, file_name.replace(".csv", "").replace(".", "_"))
  loaded = False
  output = None
      
  if use_cache and os.path.exists(f"{clean_name}_{annotation}_word.npy"):
    verbosity("Attempting to load from cache", debug)
    try:
      word_vectors = np.load(f"{clean_name}_{annotation}_word.npy", allow_pickle=True)
      char_vectors = np.load(f"{clean_name}_{annotation}_char.npy", allow_pickle=True)
      case_vectors = np.load(f"{clean_name}_{annotation}_case.npy", allow_pickle=True)
      label_vectors = np.load(f"{clean_name}_{annotation}_label.npy", allow_pickle=True)
      output = [word_vectors, char_vectors, case_vectors, label_vectors]
      loaded = True
      verbosity("Loaded successfully", debug)
    except:
      verbosity("Loading failed",debug)
      loaded = False
  
  if not loaded:
    if annotation == "BIO":
      verbosity(f"Loading raw data file to process labels: {file_name}", debug)
      checkpoint = time.time()  
      header = 0
      if 'onto' in file_name:
        header = None
      data = pd.read_csv(os.path.join(data_src, file_name), sep="\t",  quoting=csv.QUOTE_NONE, header=header, skip_blank_lines=False, engine='python', names =['token', 'pos', 'tree', 'label'])
      verbosity(f"Parsed data loaded: {time.time()-checkpoint} s", debug)

      # see if prior row was a newline
      data['prior'] = data.token.shift(1)
      data['prior_pos'] = data.pos.shift(1)
      # drop empty rows
      data = data.loc[~data.token.isnull()]
      data.prior = data.apply(checkPrior, axis=1)
      data['phrase'] = data.prior.cumsum()

      verbosity("Processing data into phrase vectors", debug)
      verbosity("Step 1: Translating to indexes", debug)
      checkpoint = time.time()
      data['word_ix'] = data.token.apply(get_word_ix)
      data['case_ix'] = data.token.apply(get_casing_ix)
      data['label_ix'] = data.label.apply(lambda x: label_to_ix[x])
      verbosity(f"Step 1: Translated to indexes complete: {time.time()-checkpoint} s", debug)

      verbosity("Step 2: Creating phrase vectors", debug)
      verbosity("Step 2a: Aggregating phrases", debug)
      checkpoint = time.time()
      phrase_vectors = data.groupby('phrase').agg({'token':list, 'word_ix': list, 'case_ix': list, 'label_ix': list})
      verbosity(f"Step 2a: {time.time()-checkpoint} s", debug)

      verbosity("Step 2b: Padding word vectors", debug)
      checkpoint = time.time()
      phrase_vectors['word_vector'] = phrase_vectors.word_ix.apply(lambda x: pad_truncate(x, SENTENCE_WIDTH, word_to_ix[PAD_WORD]))
      verbosity(f"Step 2b: {time.time()-checkpoint} s", debug)

      verbosity("Step 2c: Creating and padding character vectors", debug)
      checkpoint = time.time()
      phrase_vectors['char_vector'] = phrase_vectors.word_vector.apply(lambda x: phrase2char(x))
      verbosity(f"Step 2c: {time.time()-checkpoint} s", debug)

      verbosity(f"Step 2d: Padding case vectors", debug)
      checkpoint = time.time()
      phrase_vectors['case_vector'] = phrase_vectors.case_ix.apply(lambda x: pad_truncate(x, SENTENCE_WIDTH, case_to_ix[CASE_PAD_SYMBOL]))
      verbosity(f"Step 2d: {time.time()-checkpoint}", debug)

      verbosity("Step 2e: Padding label vectors", debug)
      checkpoint = time.time()
      phrase_vectors['label_vector'] = phrase_vectors.label_ix.apply(lambda x: np.expand_dims(pad_truncate(x, SENTENCE_WIDTH, label_to_ix[LABEL_PAD_SYMBOL]), -1))
      verbosity(f"Step 2e: {time.time()-checkpoint} s", debug)

      phrase_vectors.drop(columns=['word_ix', 'case_ix', 'label_ix'], inplace=True)

      output = [np.stack(phrase_vectors['word_vector'].values), np.stack(phrase_vectors['char_vector'].values), np.stack(phrase_vectors['case_vector'].values), np.stack(phrase_vectors['label_vector'].values)]
    
    elif annotation == "BILOU":
      verbosity(f"Loading raw data file to process labels: {file_name}", debug)
      checkpoint = time.time()  
      data = pd.read_csv(os.path.join(data_src, file_name), header=0, index_col=0,skip_blank_lines=False, engine='python')
      data = data[data.x.str.len() > 2]
      verbosity(f"Data loaded: {time.time()-checkpoint} s", debug)

      verbosity("Processing data into sentence vectors", debug)
      verbosity("Step 1: Translating to indexes", debug)
      checkpoint = time.time()
      data.x = data.x.apply(ast.literal_eval)
      data.y = data.y.apply(ast.literal_eval)
      data['word_vector'] = data.x.apply(lambda x: pad_truncate(sent_to_word_ix(x), SENTENCE_WIDTH, word_to_ix[PAD_WORD]))
      data['case_vector'] = data.x.apply(lambda x: pad_truncate(sent_to_casing_ix(x), SENTENCE_WIDTH, case_to_ix[CASE_PAD_SYMBOL]))
      data['label_vector'] = data.y.apply(lambda x: pad_truncate(sent_to_label_ix(x), SENTENCE_WIDTH, label_to_ix[LABEL_PAD_SYMBOL]))
      print("Processing character vectors")
      data['char_vector'] = data.word_vector.apply(lambda x: phrase2char(x))
      verbosity(f"Step 1: Translated to indexes complete: {time.time()-checkpoint} s", debug)

      output = [np.stack(data["word_vector"].values), np.stack(data["char_vector"].values), np.stack(data["case_vector"].values), np.expand_dims(np.stack(data["label_vector"].values),-1)]
    
    else:
      raise Exception(f"unkown annotation: {annotation}")
    
    if save_cache:
      verbosity("Saving data to disk", debug)
      checkpoint = time.time()
      # saving in multi parts because training data causes a memory error
      np.save(f"{clean_name}_{annotation}_word", output[0], allow_pickle=True)
      np.save(f"{clean_name}_{annotation}_char", output[1], allow_pickle=True)
      np.save(f"{clean_name}_{annotation}_case", output[2], allow_pickle=True)
      np.save(f"{clean_name}_{annotation}_label", output[3], allow_pickle=True)
      verbosity(f"Saved to disk: {time.time()-checkpoint} s", debug)
  
  return output

## Prepare domain data

In [7]:
real_data = preprocess_data(real_file, annotation_type, use_cache=False,save_cache=False)
print(real_data[0].shape, real_data[1].shape, real_data[2].shape, real_data[3].shape)

Loading raw data file to process labels: bios-tagged-final-flat_bilou.csv
Data loaded: 0.013188838958740234 s
Processing data into sentence vectors
Step 1: Translating to indexes
Processing character vectors
Step 1: Translated to indexes complete: 0.7101879119873047 s
(708, 256) (708, 256, 52) (708, 256) (708, 256, 1)


In [0]:
def train_dev_test_split(data, test_size, train_size):
  all_choices = np.arange(0,data[0].shape[0])
  test_choices = np.random.choice(a=all_choices, size=int(np.round(data[0].shape[0]*test_size)), replace=False)
  real_test_data = [data[0][test_choices], data[1][test_choices], data[2][test_choices], data[3][test_choices]]
  not_test_choices = np.setdiff1d(all_choices,test_choices)
  not_test_data = data[0][not_test_choices], data[1][not_test_choices], data[2][not_test_choices], data[3][not_test_choices]
  train_choices = np.random.randint(0,not_test_data[0].shape[0], size=int(np.round(not_test_data[0].shape[0]*train_size)))
  train_choices = np.random.choice(a=not_test_choices, size=int(np.round(not_test_choices.shape[0]*train_size)), replace=False)
  real_train_data = [data[0][train_choices], data[1][train_choices], data[2][train_choices], data[3][train_choices]]
  dev_choices = np.setdiff1d(not_test_choices, train_choices)
  real_dev_data = [data[0][dev_choices], data[1][dev_choices], data[2][dev_choices], data[3][dev_choices]]

  print(f"Original shape: {data[0].shape}")
  print(f"Train shape: {real_train_data[0].shape}")
  print(f"Dev shape: {real_dev_data[0].shape}")
  print(f"Test shape: {real_test_data[0].shape}")

  return real_test_data, real_dev_data, real_train_data

In [9]:
rtrain, rdev, rtest = train_dev_test_split(real_data, 0.435, 0.75)


Original shape: (708, 256)
Train shape: (300, 256)
Dev shape: (100, 256)
Test shape: (308, 256)


# Model Building

In [0]:
# https://github.com/mxhofer/Named-Entity-Recognition-BidirectionalLSTM-CNN-CoNLL/blob/master/nn_CoNLL.ipynb

def buildModel(labels, wordEmbeddings, caseEmbeddings, characterEmbeddings=None):
  
  # character - input
  character_input = Input(shape=(None, WORD_LENGTH,), name="Character_input")
  embed_char_out = TimeDistributed(
      Embedding(CHAR_VOCAB,
                CHAR_EMBEDDING_DIM,
                embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)),name="Character_embedding")(character_input)

  dropout = Dropout(DROPOUT)(embed_char_out)

  ## character - CNN
  conv1d_out = TimeDistributed(Conv1D(kernel_size=CONV_SIZE,
                                      filters=CONV_FILTERS,
                                      padding='same',
                                      activation='tanh',
                                      strides=CONV_STRIDE), name="Convolution")(dropout)
  maxpool_out = TimeDistributed(MaxPooling1D(CONV_WINDOW), name="Maxpool")(conv1d_out)
  char = TimeDistributed(Flatten(), name="Flatten")(maxpool_out)
  char = Dropout(DROPOUT)(char)

  # word - input
  words_input = Input(shape=(None,), dtype='int32', name='words_input')
  words = Embedding(input_dim=wordEmbeddings.shape[0],
                    output_dim=wordEmbeddings.shape[1],
                    weights=[wordEmbeddings],
                    trainable=False)(words_input)

  # case - input
  casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
  casing = Embedding(input_dim=caseEmbeddings.shape[0],
                     output_dim=caseEmbeddings.shape[1],
                     weights=[caseEmbeddings],
                     trainable=False)(casing_input)
  
  # character + word + case -> biLSTM
  output = concatenate([words, casing, char])
  output = Bidirectional(LSTM(LSTM_STATE_SIZE, 
                              return_sequences=True, 
                              dropout=DROPOUT,
                              recurrent_dropout=RECURRENT_DROPOUT
                             ), name="biLSTM")(output)
  
  # output
  output = TimeDistributed(Dense(len(labels), activation='softmax'),name="softmax")(output)

  # finalize
  model = Model(inputs=[words_input, character_input, casing_input], outputs=[output])

  model.compile(loss='sparse_categorical_crossentropy', optimizer=Nadam())
  
  return model

# Metrics

In [0]:
def get_metrics(model, data, save_name=None):
  predictions = model.predict(data[:3])
  y = data[3].reshape(data[3].shape[0],data[3].shape[1])
  
  pf = np.argmax(predictions, axis=2).flatten()
  af = data[3].flatten()
  
  metrics = []
  metrics = pd.DataFrame(columns=['Label', 'Support', 'Precision', 'Recall', "F1"])
  for i, label in enumerate(labels):
    support = np.where(af == i)
    tp = np.sum(pf[support] == af[support])

    precision = None
    if pf[np.where(pf == i)].shape[0] == 0:
      precision = 0.0
    else:
      precision = tp/pf[np.where(pf == i)].shape[0]
      
    recall = tp/af[support].shape[0]
    
    f1 = None
    if precision + recall == 0:
      f1 = 0
    else:
      f1 = 2*precision*recall/(precision+recall)

    metrics = metrics.append({'Label': ix_to_label[i], 'Support':af[support].shape[0], 'Precision': precision, 'Recall':recall, 'F1':f1}, ignore_index=True)
  
  metrics = metrics.append({'Label': 'micro',
                  'Support': metrics.Support.sum(),
                  'Precision': (metrics.Precision*metrics.Support/pf.shape[0]).sum(),
                  'Recall': (metrics.Recall*metrics.Support/pf.shape[0]).sum(),
                  'F1': (metrics.F1*metrics.Support/pf.shape[0]).sum()
                           },
                 ignore_index=True)
  metrics = metrics.set_index('Label')
  if save_name is not None:
    print("saving")
    metrics.to_csv(os.path.join(model_dir, save_name))
  display(metrics)
  return predictions
 

In [12]:
print("Building model")
for size in real_training_sizes:
  new_model = buildModel(labels, words, cases)
  if size == np.min(real_training_sizes):
    print(new_model.summary())
    
  t_word = rtrain[0][:size]
  t_char = rtrain[1][:size]
  t_case = rtrain[2][:size]
  t_labels = rtrain[3][:size]
  
  print(new_model.evaluate(rdev[:3], rdev[3]))
  new_model.fit([t_word, t_char, t_case], t_labels,
              validation_data = (rdev[:3], rdev[3]),
              epochs=EPOCHS,
              initial_epoch=0,
              batch_size=10,
              callbacks=[EarlyStopping(min_delta=0), ModelCheckpoint(model_path)])
  test_pred = get_metrics(new_model, rtest, save_name=model_name + f'_{size}.csv')


W0803 07:59:40.898433 139854150084480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 07:59:40.903131 139854150084480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 07:59:40.904549 139854150084480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Building model


W0803 07:59:40.930281 139854150084480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0803 07:59:40.941917 139854150084480 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0803 07:59:40.996485 139854150084480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0803 07:59:41.051987 139854150084480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tens

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Character_input (InputLayer)    (None, None, 52)     0                                            
__________________________________________________________________________________________________
Character_embedding (TimeDistri (None, None, 52, 30) 3000        Character_input[0][0]            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, 52, 30) 0           Character_embedding[0][0]        
__________________________________________________________________________________________________
Convolution (TimeDistributed)   (None, None, 52, 30) 2730        dropout_1[0][0]                  
__________________________________________________________________________________________________
Maxpool (T

W0803 07:59:45.300086 139854150084480 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars


,Support,Precision,Recall,F1
Label,,,,
B-DATE,145,0.737179,0.793103,0.764120
B-GPE,6,0.000000,0.000000,0.000000
B-LAW,2,0.000000,0.000000,0.000000
B-MONEY,3,0.000000,0.000000,0.000000
B-NORP,0,0.000000,NaN,NaN
B-ORG,381,0.765487,0.454068,0.570016
B-PERCENT,0,0.000000,NaN,NaN
B-PERSON,43,0.960000,0.558140,0.705882
B-PRODUCT,7,0.000000,0.000000,0.000000


100/100 [==============================] - 4s 37ms/step
3.679733295440674
Train on 300 samples, validate on 100 samples
Epoch 1/20
300/300 [==============================] - 38s 127ms/step - loss: 0.4851 - val_loss: 0.1340
Epoch 2/20
300/300 [==============================] - 34s 115ms/step - loss: 0.1182 - val_loss: 0.0947
Epoch 3/20
300/300 [==============================] - 34s 114ms/step - loss: 0.0875 - val_loss: 0.0710
Epoch 4/20
300/300 [==============================] - 34s 114ms/step - loss: 0.0675 - val_loss: 0.0568
Epoch 5/20
300/300 [==============================] - 34s 113ms/step - loss: 0.0573 - val_loss: 0.0499
Epoch 6/20
300/300 [==============================] - 34s 115ms/step - loss: 0.0459 - val_loss: 0.0423
Epoch 7/20
300/300 [==============================] - 34s 113ms/step - loss: 0.0401 - val_loss: 0.0382
Epoch 8/20
300/300 [==============================] - 34s 113ms/step - loss: 0.0361 - val_loss: 0.0349
Epoch 9/20
300/300 [==============================] - 34

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars


,Support,Precision,Recall,F1
Label,,,,
B-DATE,145,0.952055,0.958621,0.955326
B-GPE,6,1.000000,0.166667,0.285714
B-LAW,2,0.000000,0.000000,0.000000
B-MONEY,3,0.000000,0.000000,0.000000
B-NORP,0,0.000000,NaN,NaN
B-ORG,381,0.811414,0.858268,0.834184
B-PERCENT,0,0.000000,NaN,NaN
B-PERSON,43,0.973684,0.860465,0.913580
B-PRODUCT,7,1.000000,0.142857,0.250000
